# Projeto de Clasificação IV

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import graphviz

In [2]:
uri = "https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv"
dados = pd.read_csv(uri)
dados.head()

,Unnamed: 0,mileage_per_year,model_year,price,sold
0,0,21801,2000,30941.02,yes
1,1,7843,1998,40557.96,yes
2,2,7109,2006,89627.50,no
3,3,26823,2015,95276.14,no
4,4,7935,2014,117384.68,yes


In [3]:
a_renomear = {
    'mileage_per_year': 'milhas_por_ano',
    'model_year': 'ano_do_modelo',
    'price': 'preco',
    'sold': 'vendido'
}
dados = dados.rename(columns = a_renomear)
dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido
0,0,21801,2000,30941.02,yes
1,1,7843,1998,40557.96,yes
2,2,7109,2006,89627.50,no
3,3,26823,2015,95276.14,no
4,4,7935,2014,117384.68,yes


In [4]:
a_trocar = {
    'no': 0,
    'yes': 1
}

dados.vendido = dados.vendido.map(a_trocar)
dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido
0,0,21801,2000,30941.02,1
1,1,7843,1998,40557.96,1
2,2,7109,2006,89627.50,0
3,3,26823,2015,95276.14,0
4,4,7935,2014,117384.68,1


In [5]:
ano_atual = datetime.today().year

dados['idade_do_modelo'] = ano_atual - dados.ano_do_modelo
dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo
0,0,21801,2000,30941.02,1,22
1,1,7843,1998,40557.96,1,24
2,2,7109,2006,89627.50,0,16
3,3,26823,2015,95276.14,0,7
4,4,7935,2014,117384.68,1,8


In [6]:
dados['km_por_ano'] = (dados.milhas_por_ano * 1.60934).round(2)
dados.head()

,Unnamed: 0,milhas_por_ano,ano_do_modelo,preco,vendido,idade_do_modelo,km_por_ano
0,0,21801,2000,30941.02,1,22,35085.22
1,1,7843,1998,40557.96,1,24,12622.05
2,2,7109,2006,89627.50,0,16,11440.80
3,3,26823,2015,95276.14,0,7,43167.33
4,4,7935,2014,117384.68,1,8,12770.11


In [7]:
# Apagando as colunas não utilizadas
dados = dados.drop(columns = ['Unnamed: 0', 'milhas_por_ano', 'ano_do_modelo'], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,22,35085.22
1,40557.96,1,24,12622.05
2,89627.50,0,16,11440.80
3,95276.14,0,7,43167.33
4,117384.68,1,8,12770.11


In [8]:
x = dados[['preco', 'idade_do_modelo', 'km_por_ano']]
y = dados['vendido']

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

SEED = 50
np.random.seed(SEED)

x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    random_state = SEED, 
                                                    test_size = 0.25,
                                                    stratify = y)

# Treinando o modelo
modelo = LinearSVC()
modelo.fit(x_train, y_train)

# Aplicando o modelo nos dados de teste
previsoes = modelo.predict(x_test)

# Verificando a ACURÁCIA do modelo
acuracia = accuracy_score(y_test, previsoes)
print(f'A acurácia apurada foi de {(acuracia*100).round(2)}%')

C:\Users\Fabiano\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


A acurácia apurada foi de 58.0%


In [9]:
dummy_stratified = DummyClassifier()
dummy_stratified.fit(x_train, y_train)

# Aplicando o modelo nos dados de teste
previsoes = dummy_stratified.predict(x_test)
print(previsoes)

# Verificando a ACURÁCIA do modelo
acuracia1 = accuracy_score(y_test, previsoes)
print(f'A acurácia do dummy_stratified foi de {(acuracia1*100).round(2)}%')

[1 1 1 ... 1 1 1]
A acurácia do dummy_stratified foi de 58.0%


In [10]:
dummy_most_frequent = DummyClassifier(strategy='most_frequent')
dummy_most_frequent.fit(x_train, y_train)

# Aplicando o modelo nos dados de teste
dummy_most_frequent.score(x_test, y_test)

# Verificando a ACURÁCIA do modelo
# acuracia2 = accuracy_score(y_test, previsoes)
# print(f'A acurácia do dummy_most_frequent foi de {(acuracia2*100).round(2)}%')

0.58

In [11]:
SEED = 5
np.random.seed(SEED)

x_train_old, x_test_old, y_train, y_test = train_test_split(x, y, 
                                                    random_state = SEED, 
                                                    test_size = 0.25,
                                                    stratify = y)

print(f'Treinamos o modelo com {len(x_train)} elementos e testaremos com {len(x_test)} elementos!\n')

scaler = StandardScaler()
scaler.fit(x_train_old)
x_train = scaler.transform(x_train_old)
x_test = scaler.transform(x_test_old)

# Treinando o modelo
modelo = SVC()
modelo.fit(x_train, y_train)

# Aplicando o modelo nos dados de teste
previsoes = modelo.predict(x_test)

# Verificando a ACURÁCIA do modelo
acuracia = accuracy_score(y_test, previsoes)
print(f'A acurácia apurada foi de {(acuracia*100).round(2)}%')

Treinamos o modelo com 7500 elementos e testaremos com 2500 elementos!

A acurácia apurada foi de 77.48%


## Árvores de Decisão

In [21]:
SEED = 5
np.random.seed(SEED)

x_train_old, x_test_old, y_train, y_test = train_test_split(x, y, 
                                                    test_size = 0.25,
                                                    stratify = y)

print(f'Treinamos o modelo com {len(x_train)} elementos e testaremos com {len(x_test)} elementos!\n')

scaler = StandardScaler()
scaler.fit(x_train_old)
x_train = scaler.transform(x_train_old)
x_test = scaler.transform(x_test_old)

# Treinando o modelo
modelo = DecisionTreeClassifier(max_depth=4)
modelo.fit(x_train, y_train)

# Aplicando o modelo nos dados de teste
previsoes = modelo.predict(x_test)

# Verificando a ACURÁCIA do modelo
acuracia = accuracy_score(y_test, previsoes)
print(f'A acurácia apurada foi de {(acuracia*100).round(2)}%')

Treinamos o modelo com 7500 elementos e testaremos com 2500 elementos!

A acurácia apurada foi de 79.28%


In [ ]:
# Gerando uma visualização gráfica do modelo

dot_data = export_graphviz(modelo, out_file=None)
grafico = graphviz.Source(dot_data)
grafico